In [11]:
import pandas
import numpy as np
import yfinance as yf

## MACD (Moving Average Convergence/Divergence) oscillators

In [17]:
# Define the ticker symbol
tickerSymbol = 'AAPL'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
df = tickerData.history(period='1d', start='2021-1-1', end='2023-1-1')

df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-01-04 00:00:00-05:00,131.202835,131.291270,124.560150,127.164162,143301900,0.0,0.0
2021-01-05 00:00:00-05:00,126.653200,129.453747,126.201177,128.736404,97664900,0.0,0.0
2021-01-06 00:00:00-05:00,125.503489,128.775701,124.186740,124.402924,155088000,0.0,0.0
2021-01-07 00:00:00-05:00,126.132379,129.345634,125.641056,128.647949,109578200,0.0,0.0
2021-01-08 00:00:00-05:00,130.131742,130.328283,127.969924,129.758347,105158200,0.0,0.0


In [18]:
# Calculate Short Term Exponential Moving Average
ShortEMA = df.Close.ewm(span=12, adjust=False).mean() 
# Calculate Long Term Exponential Moving Average
LongEMA = df.Close.ewm(span=26, adjust=False).mean() 
# Calculate MACD line
MACD = ShortEMA - LongEMA
# Calculate Signal line
signal = MACD.ewm(span=9, adjust=False).mean()

In [19]:
df['MACD'] = MACD
df['Signal Line'] = signal

In [21]:
def buy_sell(signal):
    Buy = []
    Sell = []
    flag = -1

    for i in range(0, len(signal)):
        if signal['MACD'][i] > signal['Signal Line'][i]:
            Sell.append(np.nan)
            if flag != 1:
                Buy.append(signal['Close'][i])
                flag = 1
            else:
                Buy.append(np.nan)
        elif signal['MACD'][i] < signal['Signal Line'][i]:
            Buy.append(np.nan)
            if flag != 0:
                Sell.append(signal['Close'][i])
                flag = 0
            else:
                Sell.append(np.nan)
        else:
            Buy.append(np.nan)
            Sell.append(np.nan)

    return (Buy, Sell)

In [31]:
a = buy_sell(df)
buy_signal = a[0]
sell_signal = a[1]
buy_signal = [x for x in buy_signal if x is not np.nan]
sell_signal = [x for x in sell_signal if x is not np.nan]

In [32]:
print(buy_signal)
print(sell_signal)

[128.7364044189453, 128.64794921875, 128.61843872070312, 129.7386932373047, 122.02027893066406, 124.11040496826172, 148.27694702148438, 149.89605712890625, 141.9288330078125, 151.7560577392578, 172.6374969482422, 162.33413696289062, 148.35572814941406, 137.0832977294922, 141.38389587402344, 148.86790466308594]
[124.40292358398438, 126.56475830078125, 126.6728286743164, 132.64730834960938, 131.3594970703125, 143.3331756591797, 144.49575805664062, 147.07249450683594, 146.24900817871094, 169.20668029785156, 167.18495178222656, 168.38278198242188, 130.74813842773438, 166.3625946044922, 137.61082458496094, 140.38531494140625]


In [35]:
# Assume we buy and sell at the close of the day
stock_Return = []
for i in range(0, len(buy_signal)):
    stock_Return.append(sell_signal[i] - buy_signal[i])
    
print(stock_Return)

[-4.3334808349609375, -2.08319091796875, -1.9456100463867188, 2.9086151123046875, 9.339218139648438, 19.22277069091797, -3.78118896484375, -2.8235626220703125, 4.3201751708984375, 17.45062255859375, -5.452545166015625, 6.04864501953125, -17.607589721679688, 29.279296875, -3.7730712890625, -8.482589721679688]


In [37]:
print("Cumulative Return : ", np.sum(stock_Return))

Cumulative Return :  38.28651428222656


## RSI (Relative Strength Index)  

In [38]:
def calculate_RSI(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]

    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

df['RSI'] = calculate_RSI(df['Close'], 14)

In [39]:
def buy_sell_RSI(signal, upper=70, lower=30):
    Buy = []
    Sell = []

    for i in range(0, len(signal)):
        if signal['RSI'][i] > upper:
            Sell.append(signal['Close'][i])
            Buy.append(np.nan)
        elif signal['RSI'][i] < lower:
            Buy.append(signal['Close'][i])
            Sell.append(np.nan)
        else:
            Buy.append(np.nan)
            Sell.append(np.nan)

    return (Buy, Sell)

In [42]:
a = buy_sell_RSI(df)
buy_signal = a[0]
sell_signal = a[1]
buy_signal = [x for x in buy_signal if x is not np.nan]
sell_signal = [x for x in sell_signal if x is not np.nan]

In [43]:
# Assume we buy and sell at the close of the day
stock_Return = []
for i in range(0, len(buy_signal)):
    stock_Return.append(sell_signal[i] - buy_signal[i])
    
print(stock_Return)

[21.371788024902344, 21.341880798339844, 2.3903961181640625, 9.67376708984375]


In [44]:
print("Cumulative Return : ", np.sum(stock_Return))

Cumulative Return :  54.77783203125
